In [1]:
import os
import csv
import keras
import pickle
import random
import numpy as np
import pandas as pd 
import networkx as nx
from time import time
from numpy import dot
import tensorflow as tf
from numpy import unique
from numpy import loadtxt
from keras import Sequential
from numpy.linalg import norm
from keras.layers import LSTM
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from itertools import combinations
from tensorflow.keras import Model
from sklearn.utils import resample
from keras.models import Sequential
from sklearn.metrics import roc_auc_score
from sklearn.naive_bayes import GaussianNB
from tensorflow.keras.optimizers import Adam
from catboost import CatBoostClassifier, Pool
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import average_precision_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix,accuracy_score
from tensorflow.keras.layers import Lambda,Input,Conv1D,Dense,Dropout
from tensorflow.keras.layers import GlobalAveragePooling1D,MaxPooling1D,Flatten
from sklearn.metrics import f1_score,accuracy_score,precision_score,recall_score

from dynamicgem.embedding.dynAERNN import DynAERNN
from dynamicgem.evaluation import visualize_embedding as viz
from dynamicgem.visualization import plot_dynamic_sbm_embedding
from dynamicgem.graph_generation import dynamic_SBM_graph as sbm
from dynamicgem.utils import graph_util, plot_util, dataprep_util
from dynamicgem.evaluation import evaluate_graph_reconstruction as gr

random.seed(7)

In [2]:
class My_Approach:
    
    def __init__(self, dim, history, lookback, ae_series_len, series_len, subgraphs, samp_id):
        self.dim = dim
        self.history = history
        self.lookback = lookback
        self.ae_series_len = ae_series_len
        self.series_len = series_len
        self.subgraphs = subgraphs
#         self.sample = sample
        self.samp_id = samp_id
        
#     def subgraph_extraction(self):
#         subgraphs = []
#         for g in self.graphs:
#             subgraphs.append(g.subgraph(self.sample))
#         return subgraphs
    
    def emb_generate(self, subgraphs):
        embs = []
        embedding = DynAERNN(d = self.dim,
            beta           = 5,
            n_prev_graphs  = self.lookback,
            nu1            = 1e-6,
            nu2            = 1e-6,
            n_aeunits      = [500, 300],
            n_lstmunits    = [500,self.dim],
            rho            = 0.3,
            n_iter         = 150,
            xeta           = 1e-3,
            n_batch        = 100,)
        for temp_var in range(self.lookback+1, self.ae_series_len+1):
            emb, _ = embedding.learn_embeddings(subgraphs[:temp_var])
            embs.append(emb)
        return embs
    
    def generate_x_y_index(self, graph_):
        x = []
        y = []
        pos_links = list(graph_.edges)
        num_pos_links = len(pos_links)
        graph_complement = nx.complement(graph_)
        graph_complement_links = list(graph_complement.edges)
        neg_links = random.sample(graph_complement_links, num_pos_links)
        for i in range(num_pos_links):
            temp = list(pos_links[i])
            temp_2 = [list(graph_.nodes).index(temp[0]),list(graph_.nodes).index(temp[1])]
            x.append(temp_2)
            y.append(1)
        for i in range(num_pos_links):
            temp = list(neg_links[i])
            temp_2 = [list(graph_.nodes).index(temp[0]),list(graph_.nodes).index(temp[1])]
            x.append(temp_2)
            y.append(0)    
        data = list(zip(x, y))
        random.shuffle(data)
        x, y = zip(*data)
        return x, y
    
    def generate_x_y(self, loc, ind):
        pred_locs = []
        for i in (ind):
            pred_locs.append([loc[i[0]],loc[i[1]]])
        return pred_locs
    
    def score_(self, y, pred_sim):
        print('AUROC-SCORE:', roc_auc_score(y, pred_sim))
        print('AUPRC-SCORE:', average_precision_score(y, pred_sim))
    
    def similarity_(self, x):
        dist = []
        x = np.array(x)
        for i in range(len(x)):
            temp = np.linalg.norm(x[i][0]-x[i][1])
            dist.append(temp)
        max_ = max(dist)
        min_ = min(dist)
        dist = [(dist[i]-min_)/(max_-min_) for i in range(len(dist))]
        sim = [1-dist[i] for i in range(len(dist))]
#         name = 'sim_mp'+str(self.samp_id)+'_'+str(self.dim)+'_'+str(self.series_len)+'_'+str(self.history)
#         path = '/Users/sanaz/Desktop/data/My_Approach/'+str(name)+'.npy'
#         np.save(path, sim)
        return sim
    
    def raw_baseline(self, emb, ind, y):
        predicted_sim = self.similarity_(x)
        print('Raw Embedding Performance With Setting: dim = ')
        score_(y, predicted_sim)
        
    def vel_acc(self, data, vel_model):
        shape_ = list(np.array(data).shape)[1]
        vel = []
        for i in range(len(data)-1):
            vel.append(data[i+1]-data[i])
        vel_reshaped = np.array(vel).reshape(shape_,len(vel),self.dim) 
    
        train_x = []
        train_y = []
        temp = [[] for i in range(len(vel)-self.history)]
        test_x = []

        num_nodes = shape_
        for i in range(num_nodes):
            j_temp = 0
            for j in range(len(vel)-self.history):
                train_x.append(vel_reshaped[i][j:j+self.history])
                train_y.append(vel_reshaped[i][j+self.history])
                temp[j].append(vel_reshaped[i][j:j+self.history])
                j_temp = j
            test_x.append(vel_reshaped[i][j_temp+1: ])
    
        train_x, train_y, test_x = np.array(train_x), np.array(train_y), np.array(test_x)
        temp = np.array(temp)
        vel_model.fit(train_x, train_y, epochs = 150, verbose=0)
    
        pred = []
        for i in temp:
            i = np.array(i)
            pred.append(vel_model.predict(i))
        future = vel_model.predict(test_x)
        pred.append(future)
        return pred
    
    def velocity_aggregation__(self, s_graphs,vel_series):
        num_nodes = len(list(s_graphs[0].nodes))
        agg_vel = []
        num = self.history
        if self.history > len(vel_series):
            num = len(vel_series)
        for i in range(num_nodes):
            temp = 0
            for j in range(num):
                temp = temp + (j+1)*vel_series[j][i]
            temp = 2*temp/(num*(num+1))
            agg_vel.append(temp)
        return(agg_vel)

    def loc_by_vel(self, loc, vel_series):
        #vel = velocity_aggregation__(graphs,vel_series,lookback)
        res = vel_series+loc
        return res
    
    def velocity_model(self, activation_fn='relu', ret_seq=False):
        n_units = [512,256,self.dim]
        model = Sequential()
        n_rnn_layers = len(n_units)
        return_sequences = 1
        model.add(LSTM(n_units[0],
                   input_shape=(self.history, self.dim),
                   return_sequences=return_sequences))
        for l_idx, n_unit in enumerate(n_units[1:-1]):
            model.add(LSTM(n_unit,return_sequences=True))
        if n_rnn_layers > 1:
            model.add(LSTM(n_units[-1],return_sequences=False))
        xeta,n_batch = 1e-3,100
        adam = Adam(lr=xeta, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
        model.compile(optimizer=adam, loss='mse', metrics=["mse"])
        return model
    
    def neighbor_aggregation(self,graph, loc):
        nodes_nei = []
        nodes = list(graph.nodes())
        num_nodes = len(nodes)
        for i in range(num_nodes):
            nodes_nei.append(list(graph.neighbors(nodes[i])))
        nodes_nei_indices = []
        for i in range(num_nodes):
            if len(nodes_nei[i])>0:
                temp = []
                for j in nodes_nei[i]:
                    temp.append(nodes.index(j))
                temp.append(i)
                nodes_nei_indices.append(temp)
            else:
                nodes_nei_indices.append(i)
        new_loc = []
        for i in range(num_nodes):
            if type(nodes_nei_indices[i]) == list:
                temp = [0 for k in range(self.dim)]
                for j in nodes_nei_indices[i]:
                    temp = temp + loc[j]
                temp = temp/(len(nodes_nei_indices[i])+1)
                new_loc.append(temp)
            else:
                new_loc.append(loc[nodes_nei_indices[i]])       
        return new_loc
                      
    def my_main(self):
        subgraphs = self.subgraphs
        test_subgraph = subgraphs[-1]
        subgraphs = subgraphs[:-1]
        embs = self.emb_generate(subgraphs)
        #embs = self.emb_generate(subgraphs)
        #name = 'emb_'+str(self.samp_id)+'_'+str(self.dim)+'_'+str(self.series_len)+'_'+str(self.history)
        #path_embs = str(name)+'.npy'
        #np.save(path_embs, embs)
        #path_ae = 'ae_probabilities'+str(self.samp_id)+'_'+str(self.dim)+'_'+str(self.series_len)+'_'+str(self.history)+'.npy'
        #np.save(path_ae, ae_pred)
        ind, y = self.generate_x_y_index(test_subgraph)
        
        model = self.velocity_model(activation_fn='relu', ret_seq=False)
        velocity = self.vel_acc(embs, model)
        agg_vel = self.velocity_aggregation__(subgraphs,velocity)
        locPvel = self.loc_by_vel(embs[-1], agg_vel)
        final_loc = self.neighbor_aggregation(subgraphs[-1], locPvel)
        pred_locs = self.generate_x_y(final_loc, ind)
        sim = self.similarity_(pred_locs)
        self.score_(y, sim)